# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

c:\Users\hml\.conda\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\hml\AppData\Local\Temp\ipykernel_32924\676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reporte

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

PRICE_DATA_FILE = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA_FILE, "*/*.parquet"))
# Populate the dask dataframe by reading the parquet files in the price folder
dd_px = dd.read_parquet(parquet_files).set_index("ticker")



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
import numpy as np

# Add 
# 1. Adjusted Close price (Adj_Close) same as Closing price
# 2. Lag for Close price i.e., value of Close from previous row/day (Close_Lag_1)
# 3. Lag for Adjusted Close price i.e., value of Adj_Close from previous row/day (Adj_Close_Lag_1)
# 4. Returns (returns) calculated from adjusted close price and lag for adjusted close price
# 5. hi_lo_range i.e., day's high minus low price
    
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(    
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    Adj_Close = lambda x: x['Close']  
).assign(
    Adj_Close_lag_1 = lambda x: x['Adj_Close'].shift(1)
).assign(
    returns = lambda x: x['Adj_Close'] / x['Adj_Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))


C:\Users\hml\AppData\Local\Temp\ipykernel_32924\2059855708.py:10: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
dd_feat # lazy execution

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,Adj_Close,Adj_Close_lag_1,returns,hi_lo_range
npartitions=49,,,,,,,,,,,,,,
A,"datetime64[ns, America/New_York]",float64,float64,float64,float64,int64,float64,float64,int32,float64,float64,float64,float64,float64
ABNB,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:

# There should be atleast 10 vlues to calculate rolling mean over 10 days. Note: First returns will be NaN
dd_feat['rolling_average'] = dd_feat['returns'].rolling(window=10, min_periods=10).mean()
dd_feat = dd_feat.persist() # compute rolling average in lazy manner

# Compute to actually generate data in dataframe
df = dd_feat.compute()


In [7]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,Adj_Close,Adj_Close_lag_1,returns,hi_lo_range,rolling_average
ticker,,,,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.965874,140.361975,137.686329,138.524963,1441600,0.0,0.0,2024,NaN,138.524963,NaN,NaN,2.675646,NaN
A,2024-01-03 00:00:00-05:00,137.776184,137.776184,130.857431,130.947281,2074500,0.0,0.0,2024,138.524963,130.947281,138.524963,-0.054703,6.918753,NaN
A,2024-01-04 00:00:00-05:00,130.338270,131.286726,129.978853,130.787537,2446600,0.0,0.0,2024,130.947281,130.787537,130.947281,-0.001220,1.307873,NaN
A,2024-01-05 00:00:00-05:00,129.789147,131.745974,128.411380,130.348236,1394000,0.0,0.0,2024,130.787537,130.348236,130.787537,-0.003359,3.334594,NaN
A,2024-01-08 00:00:00-05:00,129.928930,133.353375,129.599464,133.163681,1311400,0.0,0.0,2024,130.348236,133.163681,130.348236,0.021599,3.753912,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165634,128.937943,128.165634,0.006026,1.752578,0.000828
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,127.819061,128.937943,-0.008678,2.000117,0.000115
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,129.264694,127.819061,0.011310,2.237759,0.001145


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No. Its not required to convert to pandas to calculate the moving average return. The call to the 'compute' method that does the actual computation and collects results into single panda Dataframe is done after adding the rolling average using dask dataframe. 

+ Would it have been better to do it in Dask? Why?

It is better to do it in Dask when working with large datasets because dask allows for parallel processing and integeration with distributed systems.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.